In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('../excel/2024_천혜향.xlsx')
# df.describe()
df


In [ ]:
df.columns

In [ ]:
df_temp_2024_cheon = df.rename(columns={
  "1.주문자명 (지금 주문서 를 쓰고 있는 사람)": "order_name",
  "2.수령인 (귤 받는사람)": "receiver_name",
  "3.수령인 연락처\n (택배사 배송 관련 문자 발송- 번호 양식 엄수 ex/010-1234-5678 (숫자만 배열x)": "receiver_phone_number",
  "4.받는사람 주소 (시/도, 시/군/구 작성요망) - 잘 확인해주세요": "receiver_address",
  "5.선택해주세요\n **9번을 확인해주세요**": "description",
  "6.원하시는 수량 (귤 받으실 분 필요수량) ": "stock",
  "8.입금자명": "depositor_name",
  "10.요청사항 있으면 적어주세요😊\n ex)현관비번": "memo",
  "타임스탬프": "created_at",
  })
df_temp_2024_cheon.drop(["7.*선택* 보내는 분 성함과 연락처 작성\n(생략시 사장님 성함과 전화번호로 자동 발송됩니다!)", "9. \n1️⃣1박스 또는 같은 주소, 같은 상품 1박스이상 주문시-> '없음' 선택\n\n2️⃣ 2박스 이상 또는 같은 주소, 같은상품 2박스 이상 주문시-> 'ooo 주문자 주문수량 총 o 박스 입니다' 적어주세요\n\n3️⃣ 같은 주소 원하시는 상품이 각각 다를 경우-> ' ooo 주문자 주문수량 총 o 박스 입니다' 를 적은 후 주문하지 않은 상품을 다시 한번 주문서를 작성해주시길 바랍니다.\nex) 상품5kg 1박스+가정용 5kg 1박스 작성금지", "주문서를 잘 작성하였는지 다시 한번 확인부탁드립니다🙏🏻 주문해주셔서 감사합니다🙇🏻‍♀️"], axis=1, inplace=True)
df_temp_2024_cheon

In [ ]:
df_temp_2022_2 = df.rename(columns={
  "주문자명 (지금 주문서를 쓰고 있는 사람)": "order_name",
  "🍊수령인 (귤 받는사람)": "receiver_name",
  "🍊수령인 연락처 (택배사 배송 관련 문자 발송- 번호 양식 엄수 ex/010-1234-5678 (숫자만 배열x)": "receiver_phone_number",
  "🍊받는사람 주소 (시/도, 시/군/구 작성요망)": "receiver_address",
  "🍊선택해주세요": "description",
  "🍊원하시는 수량 (귤 받으실 분 필요수량) ": "stock",
  "입금자명": "depositor_name",
  "요청사항 (ex/ ooo 주문자 주문수량 총 o 박스 입니다 )": "memo",
  "타임스탬프": "created_at",
  })
df_temp_2022_2.drop(["주문해주셔서 감사합니다🙇🏻‍♀️", "*선택* 보내는사람(생략시 사장님 성함으로 자동 발송)"], axis=1, inplace=True)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_temp_2022_1
df_temp_2022_2
df_temp_2023
df_temp_2024_no
df_temp_2024_han
df_temp_2024_cheon
df_temp_2025_cheon

In [ ]:
df_sum_temp = pd.concat([df_temp_2022_1,
df_temp_2022_2,
df_temp_2023,
df_temp_2024_no,
df_temp_2024_han,
df_temp_2024_cheon,
df_temp_2025_cheon])

df_sum_temp

In [ ]:
df_unique_total = df_sum_temp.drop_duplicates(subset=['receiver_address'], keep='first')
df_unique_total

In [ ]:
df_unique_total.to_csv("../processed_data/unique_total_data.csv", index=False, encoding="utf-8-sig")

In [ ]:
df_address = df_unique_total[["created_at", "receiver_name", "receiver_address", "receiver_phone_number"]]
df_address

In [ ]:
## phone_number
required_cols = ["created_at", "receiver_name", "receiver_address", "receiver_phone_number"]

if all(col in df_address.columns for col in required_cols):
    def normalize_phone_number(phone):
        """다양한 형식의 전화번호를 '010-1234-5678' 형태로 표준화하는 함수"""
        if(type(phone) == str):
            if(len(phone) != 13):
                phone = phone.replace(" ", "")
                phone = f"{phone[:3]}-{phone[3:7]}-{phone[7:]}"
            return phone
        elif(type(phone) == int):
            phone = str(phone)
            if(len(phone) == 10):
                phone = f"0{phone[:2]}-{phone[2:6]}-{phone[6:]}"
            elif(len(phone) == 8):
                phone = f"010-{phone[:4]}-{phone[4:]}"
            else:
                print(phone)
            return phone
        return phone

    # 'receiver_phone_number' 컬럼에 위 함수를 적용합니다.
    df_address['receiver_phone_number'] = df_address['receiver_phone_number'].apply(normalize_phone_number)

    # 결과 확인
    print("--- 전화번호 전처리 후 데이터 ---")
    print(df_address.head(10).to_string())

else:
    print("오류: 필요한 컬럼이 데이터프레임에 없습니다. 컬럼명을 다시 확인해주세요.")

In [ ]:
df_address[df_address["receiver_phone_number"].str.len() != 13]

In [ ]:
df_address["created_at"]

In [ ]:
import pandas as pd

# 1. 에러를 방지하도록 수정된 함수 정의
def sanitize_timestamp(s):
    if isinstance(s, str):
        s = s.replace(' 오전 ', ' AM ')
        s = s.replace(' 오후 ', ' PM ')
        s = s.replace(' GMT+9', '')
    return s

# 2. 수정한 함수를 .apply()에 적용
# 이제 NaN이나 다른 타입 값이 있어도 에러가 나지 않습니다.
df_address['created_at'] = df_address['created_at'].apply(sanitize_timestamp)

# 3. datetime 타입으로 최종 변환
df_address['created_at'] = pd.to_datetime(df_address['created_at'], format='%Y/%m/%d %I:%M:%S %p', errors='coerce')

# 변환 결과 확인
print("\nFirst 5 rows of the converted 'created_at' column:")
print(df_address['created_at'].head())

In [ ]:
df_address

In [ ]:
df_address.to_csv("../processed_data/temp_data.csv", index=False, encoding="utf-8-sig")

In [ ]:
df_address

In [ ]:
df_address_2022_2

In [ ]:
df_test = pd.concat([df_address_2022_1, df_address_2022_2])
df_test

In [ ]:
df_created = pd.read_csv("../processed_data/temp_data.csv")
df_created

In [ ]:
df_created['created_at_dt'] = pd.to_datetime(df_created['created_at'], format='mixed', errors='coerce')


# 2. 유효한 날짜들(NaT가 아닌 값)에 대해서만 최종 문자열 형태로 변환
valid_dates = df_created['created_at_dt'].notna()
df_created.loc[valid_dates, 'timestamp_for_db'] = \
    df_created.loc[valid_dates, 'created_at_dt'].dt.strftime('%Y-%m-%d %H:%M:%S') + '.' + \
    df_created.loc[valid_dates, 'created_at_dt'].dt.microsecond.astype(str).str.zfill(6).str[:3]


# 결과 확인
print("--- 변환된 datetime 객체 (NaT 포함) ---")
print(df_created['created_at_dt'])
print("\n--- DB 저장용 최종 포맷 ---")
print(df_created['timestamp_for_db'])

In [ ]:
df_final = df_created[["timestamp_for_db", "receiver_name", "receiver_address", "receiver_phone_number"]]
df_final = df_final.rename(columns={
  "timestamp_for_db": "created_at"
})
df_final

In [ ]:
df_final.to_csv("../processed_data/total_data.csv", index=False, encoding="utf-8-sig")